In [1]:
# run the cell first for all setup

import re
import os, sys
from os import walk
import subprocess
import pandas as pd
import time
import numpy
import math
import requests
from bs4 import BeautifulSoup
import json

def getpdbseq(workpath_OriPDB,pdbfile,selectedchain):
    os.chdir(workpath_OriPDB)
    AAname3to1={'ALA':'A','VAL':'V','ILE':'I','LEU':'L','MET':'M','PHE':'F','TYR':'Y','TRP':'W','SER':'S','THR':'T','ASN':'N','GLN':'Q','GLY':'G','PRO':'P','CYS':'C','SEC':'U','ARG':'R','HIS':'H','LYS':'K','ASP':'D','GLU':'E'}
    PDBaaSEQ=[]
    PDBaaSEQ3to1=[]
    pdbread=open(pdbfile,'r')
    resinum_prev=''
    for i, pdbline in enumerate (pdbread.readlines()):
        pdbline=re.sub('\n','',pdbline)
        pdbline_elements=re.split('\s+',pdbline)
        if re.search('ATOM',pdbline_elements[0]):
            try:
                aaname=pdbline_elements[3]
                resinum=pdbline_elements[5]
                chainname=pdbline_elements[4]
                if chainname in selectedchain:

                    if resinum_prev!=resinum:
                        try:
                            PDBaaSEQ3to1.append(AAname3to1[aaname])
                            PDBaaSEQ.append(aaname)
                        except:
                            pass
                        resinum_prev=resinum
            except:
                pass
    return(PDBaaSEQ,PDBaaSEQ3to1)

def protein_important_residue(RUNALIGN_PRO,PDBaaSEQ3to1,ppfeatureselected):
    PDBaaSEQ3to1_seq=''.join(i for i in PDBaaSEQ3to1)
    RUNALIGN_seq=''.join(re.sub('-','',x) for x in RUNALIGN_PRO)
    print ('PDB: ',PDBaaSEQ3to1_seq)
    print ('ALG: ',RUNALIGN_seq)
    try:
        ininum=RUNALIGN_seq.index(PDBaaSEQ3to1_seq[0:15])
        additionseq=RUNALIGN_seq[0:ininum]

        PDBaaSEQ3to1_seq2=additionseq+PDBaaSEQ3to1_seq
        alpNUM=0
        proteinNUM=0
        aacode_color=[]

        to_print_aa=''
        to_REMARK2=''

        pdbgap=0
        while proteinNUM <(len(PDBaaSEQ3to1_seq2)):
            pdbAA=PDBaaSEQ3to1_seq2[proteinNUM]
            while RUNALIGN_PRO[alpNUM]!=pdbAA:
                if RUNALIGN_PRO[alpNUM]=='-':
                    alpNUM=alpNUM+1
                else:
                    break
            if alpNUM>(len(RUNALIGN_PRO)):
                break

            if alpNUM in ppfeatureselected:
                if RUNALIGN_PRO[alpNUM]!=PDBaaSEQ3to1_seq2[proteinNUM]:
                    pdbgap=pdbgap+1
                    proteinNUM=proteinNUM-1
                else:
                    aacode_color.append((proteinNUM+1))
                    print (' ******* feature: ',alpNUM,RUNALIGN_PRO[alpNUM],pdbAA,' protein resi: ',(proteinNUM+1),PDBaaSEQ3to1_seq2[proteinNUM])

                    to_print_aa=to_print_aa+str(PDBaaSEQ3to1_seq2[proteinNUM])+str(proteinNUM+1)+', '

                    to_REMARK2=to_REMARK2+'REMARK ******* feature: '+str(alpNUM)+str(RUNALIGN_PRO[alpNUM])+str(pdbAA)+' protein resi: '+str(proteinNUM+1)+str(PDBaaSEQ3to1_seq2[proteinNUM])+'\n'
            alpNUM=alpNUM+1
            proteinNUM=proteinNUM+1
        print (to_print_aa)
        print ('gap:',pdbgap)
    except:
        to_print_aa=''
        to_REMARK2=''
        aacode_color=['']
        pass
    return(aacode_color,PDBaaSEQ3to1_seq,RUNALIGN_seq,to_print_aa,to_REMARK2)

def pdbrewrite(workpath,workpath_OriPDB,pdbfile,aacode_color,ppfeatureselected,TopKk,selectedchain,PDBaaSEQ3to1_seq,RUNALIGN_seq,to_print_aa,to_REMARK2):
    os.chdir(workpath_OriPDB)
    pdbclean=''
    pdbread=open(pdbfile,'r')
    resinum_prev=''
    MODEL=0
    ENDMDL=0
    for i, pdbline in enumerate (pdbread.readlines()):
        pdbline=re.sub('\n','',pdbline)
        pdbline_elements=re.split('\s+',pdbline)
        
        if MODEL>2:
            continue
        if re.search('MODEL',pdbline_elements[0]):
            MODEL=MODEL+1
        if re.search('ENDMDL',pdbline_elements[0]):
            ENDMDL=ENDMDL+1
        if MODEL>2 or ENDMDL>2:
            continue
        if (re.search('          H',pdbline) or re.search('HOH ',pdbline) or re.search('Na+ ',pdbline) or re.search('Cl- ',pdbline) or re.search('SO4 ',pdbline) or re.search(' CL ',pdbline) or re.search(' MPG ',pdbline) or re.search(' NAG ',pdbline)):
            continue
        if (re.search('^ATOM',pdbline) or re.search('^TER',pdbline) or re.search('^HETATM',pdbline) or re.search('^END',pdbline) ):
            try:
                atomtype=pdbline_elements[0]
                atomnum=pdbline_elements[1]
                atomname=pdbline_elements[2]
                aaname=pdbline_elements[3]
                chainname=pdbline_elements[4]
                resinum=pdbline_elements[5]
                xyz1=pdbline_elements[6]
                xyz2=pdbline_elements[7]
                xyz3=pdbline_elements[8]
                xyz4='1.00' #pdbline_elements[9]

                if chainname not in selectedchain:
                    continue

                if re.search('color',pdbfile):
                    tempfactor=pdbline_elements[10]
                else:
                    tempfactor=0

                for selectedresi in aacode_color:
                    if str(resinum)==str(selectedresi):
                        print ('** tempfactor=TopKk **  resi:',resinum,'protein-feature: ',selectedresi)
                        tempfactor=int(TopKk) 

                atomname1=pdbline_elements[-1]

                if re.search('ATOM',atomtype):
                    pdbrewrite1=f'{atomtype}  {atomnum:>5}  {atomname:<3}{aaname:>4} {chainname:>1}{resinum:>4}    {xyz1:>8}{xyz2:>8}{xyz3:>8}  {xyz4:^4}  {tempfactor:>5}            {atomname1:>2}  \n'
                else:
                    pdbrewrite1=f'{atomtype}{atomnum:>5}  {atomname:<3}{aaname:>4} {chainname:>1}{resinum:>4}    {xyz1:>8}{xyz2:>8}{xyz3:>8}  {xyz4:^4}  {tempfactor:>5}            {atomname1:>2}  \n'

                
                pdbclean=pdbclean+pdbrewrite1

            except:
                pass
    selefeature='selefeatureTOP'+str(TopKk)
    pymol='select resi '+'+'.join(str(i) for i in aacode_color)
    pdbfile_clean=re.sub('\.pdb','',pdbfile)+'_'+'chain_'+str(selectedchain[0])+'_'+str(selefeature)+'_color.pdb'
    pdbREMARK='REMARK  MTL-AG-ATG-FS selected features: '+str(ppfeatureselected)+'\n'
    pdbREMARK=pdbREMARK+'REMARK PROseq: '+RUNALIGN_seq+' \n'
    pdbREMARK=pdbREMARK+'REMARK PDBseq: '+PDBaaSEQ3to1_seq+' \n'
    pdbREMARK=pdbREMARK+to_REMARK2
    pdbREMARK=pdbREMARK+'REMARK aacode_color: '+str(aacode_color)+'\n'
    pdbREMARK=pdbREMARK+'REMARK aacode_color_selected: '+to_print_aa+' \n'
    pdbREMARK=pdbREMARK+'REMARK pymol: '+pymol+' \n'

    pdbclean=pdbREMARK+'\n'+pdbclean+'\n'+'END'+'\n'
    
    os.chdir(workpath)
    file_w=open(pdbfile_clean,'w')
    file_w.write(pdbclean)
    file_w.close()
    return(pdbfile_clean)

def pdb_color_features(workpath,workpath_OriPDB,pdbfiles,ppcodeimportant,TopK,prot_conv):
    pdbfile=pdbfiles[0]
    proteiname=pdbfiles[1]
    selectedchain=pdbfiles[2]
    PDBaaSEQ,PDBaaSEQ3to1=getpdbseq(workpath_OriPDB,pdbfile,selectedchain)

    PROT_HUMAN=[re.sub('\s|\[|\]|\'','',x) for x in prot_conv[proteiname][4].split(',')]
    PROT_HUMAN_seq=''.join(re.sub('-','',x) for x in PROT_HUMAN)

    for k,ppfeatureselected in enumerate(ppcodeimportant):
        print ('ppfeatureselected: ',ppfeatureselected)
      
        aacode_color,PDBaaSEQ3to1_seq,RUNALIGN_seq,to_print_aa,to_REMARK2=protein_important_residue(RUNALIGN_PRO=PROT_HUMAN,PDBaaSEQ3to1=PDBaaSEQ3to1,ppfeatureselected=ppfeatureselected)
        print(aacode_color)
        pymol='select resi '+'+'.join(str(i) for i in aacode_color)
        print ('pymol: ',pymol)
        TopKk=TopK[k]
        pdbfile_out=pdbrewrite(workpath,workpath_OriPDB,pdbfile,aacode_color,ppfeatureselected,TopKk,selectedchain,PDBaaSEQ3to1_seq,RUNALIGN_seq,to_print_aa,to_REMARK2)
        print (' * ',TopK[k],'output to pdb:', pdbfile_out,'\n\n')
    return(pdbfile_out)
    
def PROclean(workpath_OriPDB,pdbfile):
    os.chdir(workpath_OriPDB)
    pdbfilenameclean='PROclean_'+re.sub('PRO_','',pdbfile)
    pdbread=open(pdbfile,'r')
    MODEL=0
    ENDMDL=0
    pdbclean=''
    for i, pdbline in enumerate (pdbread.readlines()):
        if MODEL>2:
            continue
        if re.search('^MODEL',pdbline):
            MODEL=MODEL+1
        if re.search('^ENDMDL',pdbline):
            ENDMDL=ENDMDL+1
        if MODEL<2 and ENDMDL<2:
            if (re.search('         H',pdbline) or re.search(' HOH ',pdbline) or re.search(' Na+ ',pdbline) or re.search(' Cl- ',pdbline) or re.search(' SO4 ',pdbline) or re.search(' CL ',pdbline)):
                continue
            if (re.search(' CLR ',pdbline) or re.search(' MPG ',pdbline) or re.search(' NAG ',pdbline)):
                continue
            if (re.search('^ATOM',pdbline) or re.search('^TER',pdbline) or re.search('^HETATM',pdbline) or re.search('^END',pdbline) ):
                pdbclean=pdbclean+pdbline
    file_w=open(pdbfilenameclean,'w')
    file_w.write(pdbclean)
    file_w.close()
    return(pdbfilenameclean) 




homepath=str(os.getcwd())









### CODE: Main


In [2]:

timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))
starttime=time.time()

workpath=homepath+'/colorPDB_selefeatureTOP200'

os.chdir(workpath)
ppalignfile='proteinseq20220915_align.csv'
protalign=pd.read_csv(ppalignfile,header=None ) #,names=protalign_names)
protalign1=protalign.T.set_index(1).T
prot_conv=protalign1.to_dict('list')

ppcodeimportant=[[]]
TopK=[]

filein='GPCR_to_trainAgonistAntagonist_humanalltrainconcat_selefeatureTOP200.csv'
df=pd.read_csv(filein)
for ppcode in df.columns:
    if re.search('Protein_code1',ppcode):
        ppcode2=int(re.sub('Protein_code1_','',ppcode) )
        ppcodeimportant[0].append(ppcode2)
ppcodeimportant[0].sort()
TopK.append(200)
print ('selefeatureTOP200: ',ppcodeimportant[0])

workpath_OriPDB=workpath+'/OriPDB'
os.chdir(workpath_OriPDB)

TopK2=TopK.copy()
filenames = next(walk(workpath_OriPDB), (None, None, []))[2] 
PRO_PDBs=[]
for pdbfile in filenames:
    if re.search('.pdb', pdbfile):
        if re.search('^PRO_', pdbfile):
            pdbfilenameclean=PROclean(workpath_OriPDB,pdbfile)

filenames = next(walk(workpath_OriPDB), (None, None, []))[2] 
proteinPDBs=[]
for pdbclean in filenames:
    if re.search('.pdb', pdbclean):
        if re.search('^PROclean_', pdbclean):
            proteinPDBs.append(pdbclean)
print (proteinPDBs, len(proteinPDBs))

for pdbname in proteinPDBs:
    pdbname_splits=re.split('_',pdbname)
    protein_name=pdbname_splits[1]+'_'+pdbname_splits[2]
    
    pdbfiles=[pdbname,protein_name,['A']]
    print (pdbfiles)
    pdbfile_out=pdb_color_features(workpath,workpath_OriPDB,pdbfiles,ppcodeimportant,TopK,prot_conv)
    
    pdbfiles=[pdbname,protein_name,['R']]
    print (pdbfiles)
    pdbfile_out=pdb_color_features(workpath,workpath_OriPDB,pdbfiles,ppcodeimportant,TopK,prot_conv)
    
    
    
if (time.time()-starttime)/3600 > 1 :
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
else:
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/60, ' mins \n' )


/tmp/ipykernel_962003/2888125097.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  prot_conv=protalign1.to_dict('list')


selefeatureTOP200:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 24, 25, 31, 37, 41, 45, 57, 62, 64, 69, 79, 94, 101, 104, 108, 152, 153, 155, 156, 157, 158, 159, 241, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 308, 309, 310, 311, 312, 313, 349, 350, 351, 352, 353, 354, 355, 356, 359, 422, 450, 451, 453, 458, 459, 460, 462, 463, 464, 465, 466, 467, 468, 470, 471, 472, 473, 474, 476, 477, 478, 479, 480, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 495, 499, 505, 507, 508, 518, 519, 520, 522, 531, 532, 533, 535, 536, 537, 538, 539, 540, 542, 543, 544, 545, 546, 547, 549, 554, 555, 556, 557, 560, 562, 563, 564, 594, 595, 596, 597, 640, 652, 686, 752, 759, 807, 951, 1136, 1359, 1663, 1664, 1748, 1750, 1793, 1794, 1904, 2341, 2344]
['PROclean_5HT1F_HUMAN_pdb_7EXD_366aa.pdb', 'PROclean_5HT2A_HUMAN_pdb_6WGT_200aa.pdb', 'PROclean_5HT2A_HUMAN_pdb_6A94_196aa.pdb', 'PROclean_5HT2A_HUMAN_pdb_6A93_196aa.pdb', 'PROclean_5HT1B_HUMAN

ppfeatureselected:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 24, 25, 31, 37, 41, 45, 57, 62, 64, 69, 79, 94, 101, 104, 108, 152, 153, 155, 156, 157, 158, 159, 241, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 308, 309, 310, 311, 312, 313, 349, 350, 351, 352, 353, 354, 355, 356, 359, 422, 450, 451, 453, 458, 459, 460, 462, 463, 464, 465, 466, 467, 468, 470, 471, 472, 473, 474, 476, 477, 478, 479, 480, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 495, 499, 505, 507, 508, 518, 519, 520, 522, 531, 532, 533, 535, 536, 537, 538, 539, 540, 542, 543, 544, 545, 546, 547, 549, 554, 555, 556, 557, 560, 562, 563, 564, 594, 595, 596, 597, 640, 652, 686, 752, 759, 807, 951, 1136, 1359, 1663, 1664, 1748, 1750, 1793, 1794, 1904, 2341, 2344]
PDB:  QEKNWSALLTAVVIILTIAGNILVIMAVSLEKKLQNATNYFLMSLAIADMLLGFLVMPVSMLTILYGYRWPLPSKLCAVWIYLDVLFSTASIMHLCAISLDRYVAIQNPISRFNSRTKAFLKIIAVWTISVGISMPIPVFGLSKVFKEGSCLLADDNFVLIGSFVSFFIPATIMVITYFLTIKSLQKE

ppfeatureselected:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 24, 25, 31, 37, 41, 45, 57, 62, 64, 69, 79, 94, 101, 104, 108, 152, 153, 155, 156, 157, 158, 159, 241, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 308, 309, 310, 311, 312, 313, 349, 350, 351, 352, 353, 354, 355, 356, 359, 422, 450, 451, 453, 458, 459, 460, 462, 463, 464, 465, 466, 467, 468, 470, 471, 472, 473, 474, 476, 477, 478, 479, 480, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 495, 499, 505, 507, 508, 518, 519, 520, 522, 531, 532, 533, 535, 536, 537, 538, 539, 540, 542, 543, 544, 545, 546, 547, 549, 554, 555, 556, 557, 560, 562, 563, 564, 594, 595, 596, 597, 640, 652, 686, 752, 759, 807, 951, 1136, 1359, 1663, 1664, 1748, 1750, 1793, 1794, 1904, 2341, 2344]
PDB:  
ALG:  MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWTVDSENRTNLSCEGCLSPSCLSLLHLQEKNWSALLTAVVIILTIAGNILVIMAVSLEKKLQNATNYFLMSLAIADMLLGFLVMPVSMLTILYGYRWPLPSKLCAVWIYLDVLFSTASIMHLCAISLDRYVAIQNPI

** tempfactor=TopKk **  resi: 102 protein-feature:  102
** tempfactor=TopKk **  resi: 102 protein-feature:  102
** tempfactor=TopKk **  resi: 102 protein-feature:  102
** tempfactor=TopKk **  resi: 119 protein-feature:  119
** tempfactor=TopKk **  resi: 119 protein-feature:  119
** tempfactor=TopKk **  resi: 119 protein-feature:  119
** tempfactor=TopKk **  resi: 119 protein-feature:  119
** tempfactor=TopKk **  resi: 119 protein-feature:  119
** tempfactor=TopKk **  resi: 119 protein-feature:  119
** tempfactor=TopKk **  resi: 119 protein-feature:  119
** tempfactor=TopKk **  resi: 119 protein-feature:  119
** tempfactor=TopKk **  resi: 119 protein-feature:  119
 *  200 output to pdb: PROclean_5HT1B_HUMAN_pdb_4IAQ_207aa_chain_A_selefeatureTOP200_color.pdb 


['PROclean_5HT1B_HUMAN_pdb_4IAQ_207aa.pdb', '5HT1B_HUMAN', ['R']]
ppfeatureselected:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 24, 25, 31, 37, 41, 45, 57, 62, 64, 69, 79, 94, 101, 104, 108, 152, 153, 155, 156, 157, 158, 159,

...

 *  200 output to pdb: PROclean_5HT1A_HUMAN_pdb_7E2Z_398aa_chain_A_selefeatureTOP200_color.pdb 


['PROclean_5HT1A_HUMAN_pdb_7E2Z_398aa.pdb', '5HT1A_HUMAN', ['R']]
ppfeatureselected:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 24, 25, 31, 37, 41, 45, 57, 62, 64, 69, 79, 94, 101, 104, 108, 152, 153, 155, 156, 157, 158, 159, 241, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 308, 309, 310, 311, 312, 313, 349, 350, 351, 352, 353, 354, 355, 356, 359, 422, 450, 451, 453, 458, 459, 460, 462, 463, 464, 465, 466, 467, 468, 470, 471, 472, 473, 474, 476, 477, 478, 479, 480, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 495, 499, 505, 507, 508, 518, 519, 520, 522, 531, 532, 533, 535, 536, 537, 538, 539, 540, 542, 543, 544, 545, 546, 547, 549, 554, 555, 556, 557, 560, 562, 563, 564, 594, 595, 596, 597, 640, 652, 686, 752, 759, 807, 951, 1136, 1359, 1663, 1664, 1748, 1750, 1793, 1794, 1904, 2341, 2344]
PDB:  YQVITSLLLGTLIFCAVLGNACVV

 *  200 output to pdb: PROclean_5HT1A_HUMAN_pdb_7E2Z_398aa_chain_R_selefeatureTOP200_color.pdb 


['PROclean_5HT2A_HUMAN_pdb_7RAN_339aa.pdb', '5HT2A_HUMAN', ['A']]
ppfeatureselected:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 24, 25, 31, 37, 41, 45, 57, 62, 64, 69, 79, 94, 101, 104, 108, 152, 153, 155, 156, 157, 158, 159, 241, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 308, 309, 310, 311, 312, 313, 349, 350, 351, 352, 353, 354, 355, 356, 359, 422, 450, 451, 453, 458, 459, 460, 462, 463, 464, 465, 466, 467, 468, 470, 471, 472, 473, 474, 476, 477, 478, 479, 480, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 495, 499, 505, 507, 508, 518, 519, 520, 522, 531, 532, 533, 535, 536, 537, 538, 539, 540, 542, 543, 544, 545, 546, 547, 549, 554, 555, 556, 557, 560, 562, 563, 564, 594, 595, 596, 597, 640, 652, 686, 752, 759, 807, 951, 1136, 1359, 1663, 1664, 1748, 1750, 1793, 1794, 1904, 2341, 2344]
PDB:  LLTAVVIILTIAGNILVIMAVSLE

 ******* feature:  499 V V  protein resi:  53 V
 ******* feature:  522 I I  protein resi:  59 I
 ******* feature:  539 A A  protein resi:  62 A
 ******* feature:  540 T T  protein resi:  63 T
 ******* feature:  542 L L  protein resi:  65 L
 ******* feature:  543 S S  protein resi:  66 S
 ******* feature:  555 N N  protein resi:  67 N
 ******* feature:  556 A A  protein resi:  68 A
 ******* feature:  557 F F  protein resi:  69 F
 ******* feature:  560 A A  protein resi:  72 A
 ******* feature:  562 V V  protein resi:  74 V
 ******* feature:  640 V V  protein resi:  102 V
 ******* feature:  686 Q Q  protein resi:  119 Q
 ******* feature:  1904 F F  protein resi:  342 F
['']
pymol:  select resi 
 *  200 output to pdb: PROclean_5HT1B_HUMAN_pdb_7C61_207aa_chain_A_selefeatureTOP200_color.pdb 


['PROclean_5HT1B_HUMAN_pdb_7C61_207aa.pdb', '5HT1B_HUMAN', ['R']]
ppfeatureselected:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 24, 25, 31, 37, 41, 45, 57, 62, 64, 69, 79, 94, 101, 104, 108, 152,

** tempfactor=TopKk **  resi: 36 protein-feature:  36
** tempfactor=TopKk **  resi: 36 protein-feature:  36
** tempfactor=TopKk **  resi: 36 protein-feature:  36
** tempfactor=TopKk **  resi: 36 protein-feature:  36
** tempfactor=TopKk **  resi: 36 protein-feature:  36
** tempfactor=TopKk **  resi: 36 protein-feature:  36
** tempfactor=TopKk **  resi: 36 protein-feature:  36
** tempfactor=TopKk **  resi: 36 protein-feature:  36
** tempfactor=TopKk **  resi: 36 protein-feature:  36
** tempfactor=TopKk **  resi: 37 protein-feature:  37
** tempfactor=TopKk **  resi: 37 protein-feature:  37
** tempfactor=TopKk **  resi: 37 protein-feature:  37
** tempfactor=TopKk **  resi: 37 protein-feature:  37
** tempfactor=TopKk **  resi: 37 protein-feature:  37
** tempfactor=TopKk **  resi: 37 protein-feature:  37
** tempfactor=TopKk **  resi: 37 protein-feature:  37
** tempfactor=TopKk **  resi: 40 protein-feature:  40
** tempfactor=TopKk **  resi: 40 protein-feature:  40
** tempfactor=TopKk **  resi

 *  200 output to pdb: PROclean_5HT1A_HUMAN_pdb_7E2X_398aa_chain_R_selefeatureTOP200_color.pdb 


['PROclean_5HT2A_HUMAN_pdb_6WH4_200aa.pdb', '5HT2A_HUMAN', ['A']]
ppfeatureselected:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 24, 25, 31, 37, 41, 45, 57, 62, 64, 69, 79, 94, 101, 104, 108, 152, 153, 155, 156, 157, 158, 159, 241, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 308, 309, 310, 311, 312, 313, 349, 350, 351, 352, 353, 354, 355, 356, 359, 422, 450, 451, 453, 458, 459, 460, 462, 463, 464, 465, 466, 467, 468, 470, 471, 472, 473, 474, 476, 477, 478, 479, 480, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 495, 499, 505, 507, 508, 518, 519, 520, 522, 531, 532, 533, 535, 536, 537, 538, 539, 540, 542, 543, 544, 545, 546, 547, 549, 554, 555, 556, 557, 560, 562, 563, 564, 594, 595, 596, 597, 640, 652, 686, 752, 759, 807, 951, 1136, 1359, 1663, 1664, 1748, 1750, 1793, 1794, 1904, 2341, 2344]
PDB:  LQEKNWSALLTAVVIILTIAGNIL